In [1]:
import numpy as np
import fitsio
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
import os
from astropy.table import Table, hstack
from astropy.io import fits
import tqdm
import scipy.optimize
from astropy.table import Table, vstack

In [2]:
dpi = 150
plt.rcParams['figure.dpi']= dpi
plt.rc("savefig", dpi=dpi)
plt.rc('font', size=10)
plt.rc('xtick', direction='in') 
plt.rc('ytick', direction='in')
plt.rc('xtick.major', pad=5) 
plt.rc('xtick.minor', pad=5)
plt.rc('ytick.major', pad=5) 
plt.rc('ytick.minor', pad=5)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.markersize'] = 3
plt.rc('lines', dotted_pattern = [2., 2.])

In [3]:
def mask_default(cat):
    mask0_default = (~((cat['mcal_T_noshear'][:]>2)*(cat['mcal_s2n_noshear'][:]<30)))*(cat['mcal_T_noshear'][:]<10)*(cat['mcal_s2n_noshear'][:]<1000)*(cat['mcal_s2n_noshear'][:]>10)*(cat['mcal_T_ratio_noshear'][:]>0.5)
    mask_1p_default = (~((cat['mcal_T_1p'][:]>2)*(cat['mcal_s2n_1p'][:]<30)))*(cat['mcal_T_1p'][:]<10)*(cat['mcal_s2n_1p'][:]<1000)*(cat['mcal_s2n_1p'][:]>10)*(cat['mcal_T_ratio_1p'][:]>0.5)
    mask_1m_default = (~((cat['mcal_T_1m'][:]>2)*(cat['mcal_s2n_1m'][:]<30)))*(cat['mcal_T_1m'][:]<10)*(cat['mcal_s2n_1m'][:]<1000)*(cat['mcal_s2n_1m'][:]>10)*(cat['mcal_T_ratio_1m'][:]>0.5)
    mask_2p_default = (~((cat['mcal_T_2p'][:]>2)*(cat['mcal_s2n_2p'][:]<30)))*(cat['mcal_T_2p'][:]<10)*(cat['mcal_s2n_2p'][:]<1000)*(cat['mcal_s2n_2p'][:]>10)*(cat['mcal_T_ratio_2p'][:]>0.5)
    mask_2m_default = (~((cat['mcal_T_2m'][:]>2)*(cat['mcal_s2n_2m'][:]<30)))*(cat['mcal_T_2m'][:]<10)*(cat['mcal_s2n_2m'][:]<1000)*(cat['mcal_s2n_2m'][:]>10)*(cat['mcal_T_ratio_2m'][:]>0.5)

    return mask0_default, mask_1p_default, mask_1m_default, mask_2p_default, mask_2m_default

In [5]:
with h5py.File('/project2/chihway/data/decade/metacal_test_20230323.hdf', 'r') as cat: 
    print(cat.keys())
    length = len(cat['ra'])
    print(length)

<KeysViewHDF5 ['Ncutouts_raw', 'badfrac', 'ccdnum', 'dec', 'id', 'mcal_T_1m', 'mcal_T_1p', 'mcal_T_2m', 'mcal_T_2p', 'mcal_T_noshear', 'mcal_T_ratio_1m', 'mcal_T_ratio_1p', 'mcal_T_ratio_2m', 'mcal_T_ratio_2p', 'mcal_T_ratio_noshear', 'mcal_flags', 'mcal_flux_1m', 'mcal_flux_1p', 'mcal_flux_2m', 'mcal_flux_2p', 'mcal_flux_err_1m', 'mcal_flux_err_1p', 'mcal_flux_err_2m', 'mcal_flux_err_2p', 'mcal_flux_err_noshear', 'mcal_flux_noshear', 'mcal_g_1m', 'mcal_g_1p', 'mcal_g_2m', 'mcal_g_2p', 'mcal_g_cov_1m', 'mcal_g_cov_1p', 'mcal_g_cov_2m', 'mcal_g_cov_2p', 'mcal_g_cov_noshear', 'mcal_g_noshear', 'mcal_psf_T_noshear', 'mcal_psf_g_noshear', 'mcal_s2n_1m', 'mcal_s2n_1p', 'mcal_s2n_2m', 'mcal_s2n_2p', 'mcal_s2n_noshear', 'ra', 'x', 'x_exp', 'y', 'y_exp']>
107820171


In [7]:
million = 1000000
dgamma = 2*0.01
n_iter = int(np.ceil(length/million))

R11i_1p_sum = 0
R11i_1m_sum = 0
R22i_2p_sum = 0
R22i_2m_sum = 0

R11s_1p_sum = 0
R11s_1m_sum = 0
R22s_2p_sum = 0
R22s_2m_sum = 0

R11_1p_sum = 0
R11_1m_sum = 0
R22_2p_sum = 0
R22_2m_sum = 0

mask0_count = 0
mask_1p_count = 0
mask_1m_count = 0
mask_2p_count = 0
mask_2m_count = 0

with h5py.File('/project2/chihway/data/decade/metacal_test_20230323.hdf', 'r') as cat: 
    for i in tqdm.trange(0,n_iter): 
        mcal_T_noshear = np.array(cat['mcal_T_noshear'][i*million:(i+1)*million])
        mcal_T_1p = np.array(cat['mcal_T_1p'][i*million:(i+1)*million])
        mcal_T_1m = np.array(cat['mcal_T_1m'][i*million:(i+1)*million])
        mcal_T_2p = np.array(cat['mcal_T_2p'][i*million:(i+1)*million])
        mcal_T_2m = np.array(cat['mcal_T_2m'][i*million:(i+1)*million])

        mcal_T_ratio_noshear = np.array(cat['mcal_T_ratio_noshear'][i*million:(i+1)*million])
        mcal_T_ratio_1p = np.array(cat['mcal_T_ratio_1p'][i*million:(i+1)*million])
        mcal_T_ratio_1m = np.array(cat['mcal_T_ratio_1m'][i*million:(i+1)*million])
        mcal_T_ratio_2p = np.array(cat['mcal_T_ratio_2p'][i*million:(i+1)*million])
        mcal_T_ratio_2m = np.array(cat['mcal_T_ratio_2m'][i*million:(i+1)*million])

        mcal_s2n_noshear = np.array(cat['mcal_s2n_noshear'][i*million:(i+1)*million])
        mcal_s2n_1p = np.array(cat['mcal_s2n_1p'][i*million:(i+1)*million])
        mcal_s2n_1m = np.array(cat['mcal_s2n_1m'][i*million:(i+1)*million])
        mcal_s2n_2p = np.array(cat['mcal_s2n_2p'][i*million:(i+1)*million])
        mcal_s2n_2m = np.array(cat['mcal_s2n_2m'][i*million:(i+1)*million])

        mcal_g_noshear = np.array(cat['mcal_g_noshear'][i*million:(i+1)*million])
        mcal_g_1p = np.array(cat['mcal_g_1p'][i*million:(i+1)*million])
        mcal_g_1m = np.array(cat['mcal_g_1m'][i*million:(i+1)*million])
        mcal_g_2p = np.array(cat['mcal_g_2p'][i*million:(i+1)*million])
        mcal_g_2m = np.array(cat['mcal_g_2m'][i*million:(i+1)*million])

        mask0 = (~((mcal_T_noshear>2)*(mcal_s2n_noshear<30)))*(mcal_T_noshear<10)*(mcal_s2n_noshear<1000)*(mcal_s2n_noshear>10)*(mcal_T_ratio_noshear>0.5)
        mask_1p = (~((mcal_T_1p>2)*(mcal_s2n_1p<30)))*(mcal_T_1p<10)*(mcal_s2n_1p<1000)*(mcal_s2n_1p>10)*(mcal_T_ratio_1p>0.5)
        mask_1m = (~((mcal_T_1m>2)*(mcal_s2n_1m<30)))*(mcal_T_1m<10)*(mcal_s2n_1m<1000)*(mcal_s2n_1m>10)*(mcal_T_ratio_1m>0.5)
        mask_2p = (~((mcal_T_2p>2)*(mcal_s2n_2p<30)))*(mcal_T_2p<10)*(mcal_s2n_2p<1000)*(mcal_s2n_2p>10)*(mcal_T_ratio_2p>0.5)
        mask_2m = (~((mcal_T_2m>2)*(mcal_s2n_2m<30)))*(mcal_T_2m<10)*(mcal_s2n_2m<1000)*(mcal_s2n_2m>10)*(mcal_T_ratio_2m>0.5)

        R11i_1p_sum += np.sum(mcal_g_1p[:,0][mask0])
        R11i_1m_sum += np.sum(mcal_g_1m[:,0][mask0])
        R22i_2p_sum += np.sum(mcal_g_2p[:,1][mask0])
        R22i_2m_sum += np.sum(mcal_g_2m[:,1][mask0])

        R11s_1p_sum += np.sum(mcal_g_noshear[:,0][mask_1p])
        R11s_1m_sum += np.sum(mcal_g_noshear[:,0][mask_1m])
        R22s_2p_sum += np.sum(mcal_g_noshear[:,1][mask_2p])
        R22s_2m_sum += np.sum(mcal_g_noshear[:,1][mask_2m])

        R11_1p_sum += np.sum(mcal_g_1p[:,0][mask_1p])
        R11_1m_sum += np.sum(mcal_g_1m[:,0][mask_1m])
        R22_2p_sum += np.sum(mcal_g_2p[:,1][mask_2p])
        R22_2m_sum += np.sum(mcal_g_2m[:,1][mask_2m])

        mask0_count += np.sum(mask0)
        mask_1p_count += np.sum(mask_1p)
        mask_1m_count += np.sum(mask_1m)
        mask_2p_count += np.sum(mask_2p)
        mask_2m_count += np.sum(mask_2m)

    
R11i = (R11i_1p_sum/mask0_count - R11i_1m_sum/mask0_count)/dgamma
R11s = (R11s_1p_sum/mask_1p_count - R11s_1m_sum/mask_1m_count)/dgamma
R11tot = R11i + R11s

R22i = (R22i_2p_sum/mask0_count - R22i_2m_sum/mask0_count)/dgamma
R22s = (R22s_2p_sum/mask_2p_count - R22s_2m_sum/mask_2m_count)/dgamma
R22tot = R22i + R22s

R11 = (R11_1p_sum/mask_1p_count - R11_1m_sum/mask_1m_count)/dgamma
R22 = (R22_2p_sum/mask_2p_count - R22_2m_sum/mask_2m_count)/dgamma


100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [1:21:16<00:00, 45.15s/it]


In [8]:
print(R11tot, R22tot)
print(R11, R22)

0.7356327303821948 0.7367618447379479
0.7358509828007623 0.7369249358421287


In [10]:
(0.7356327303821948-0.7358509828007623)/0.7356327303821948,\
(0.7367618447379479-0.7369249358421287)/0.7367618447379479

(-0.00029668666109262395, -0.00022136203896224783)

In [16]:
R11i_1p_err_sum = 0
R11i_1m_err_sum = 0
R22i_2p_err_sum = 0
R22i_2m_err_sum = 0

R11s_1p_err_sum = 0
R11s_1m_err_sum = 0
R22s_2p_err_sum = 0
R22s_2m_err_sum = 0

R11_1p_err_sum = 0
R11_1m_err_sum = 0
R22_2p_err_sum = 0
R22_2m_err_sum = 0


with h5py.File('/project2/chihway/data/decade/metacal_test_20230323.hdf', 'r') as cat: 
    for i in tqdm.trange(0,n_iter): 
        mcal_T_noshear = np.array(cat['mcal_T_noshear'][i*million:(i+1)*million])
        mcal_T_1p = np.array(cat['mcal_T_1p'][i*million:(i+1)*million])
        mcal_T_1m = np.array(cat['mcal_T_1m'][i*million:(i+1)*million])
        mcal_T_2p = np.array(cat['mcal_T_2p'][i*million:(i+1)*million])
        mcal_T_2m = np.array(cat['mcal_T_2m'][i*million:(i+1)*million])

        mcal_T_ratio_noshear = np.array(cat['mcal_T_ratio_noshear'][i*million:(i+1)*million])
        mcal_T_ratio_1p = np.array(cat['mcal_T_ratio_1p'][i*million:(i+1)*million])
        mcal_T_ratio_1m = np.array(cat['mcal_T_ratio_1m'][i*million:(i+1)*million])
        mcal_T_ratio_2p = np.array(cat['mcal_T_ratio_2p'][i*million:(i+1)*million])
        mcal_T_ratio_2m = np.array(cat['mcal_T_ratio_2m'][i*million:(i+1)*million])

        mcal_s2n_noshear = np.array(cat['mcal_s2n_noshear'][i*million:(i+1)*million])
        mcal_s2n_1p = np.array(cat['mcal_s2n_1p'][i*million:(i+1)*million])
        mcal_s2n_1m = np.array(cat['mcal_s2n_1m'][i*million:(i+1)*million])
        mcal_s2n_2p = np.array(cat['mcal_s2n_2p'][i*million:(i+1)*million])
        mcal_s2n_2m = np.array(cat['mcal_s2n_2m'][i*million:(i+1)*million])

        mcal_g_noshear = np.array(cat['mcal_g_noshear'][i*million:(i+1)*million])
        mcal_g_1p = np.array(cat['mcal_g_1p'][i*million:(i+1)*million])
        mcal_g_1m = np.array(cat['mcal_g_1m'][i*million:(i+1)*million])
        mcal_g_2p = np.array(cat['mcal_g_2p'][i*million:(i+1)*million])
        mcal_g_2m = np.array(cat['mcal_g_2m'][i*million:(i+1)*million])

        mask0 = (~((mcal_T_noshear>2)*(mcal_s2n_noshear<30)))*(mcal_T_noshear<10)*(mcal_s2n_noshear<1000)*(mcal_s2n_noshear>10)*(mcal_T_ratio_noshear>0.5)
        mask_1p = (~((mcal_T_1p>2)*(mcal_s2n_1p<30)))*(mcal_T_1p<10)*(mcal_s2n_1p<1000)*(mcal_s2n_1p>10)*(mcal_T_ratio_1p>0.5)
        mask_1m = (~((mcal_T_1m>2)*(mcal_s2n_1m<30)))*(mcal_T_1m<10)*(mcal_s2n_1m<1000)*(mcal_s2n_1m>10)*(mcal_T_ratio_1m>0.5)
        mask_2p = (~((mcal_T_2p>2)*(mcal_s2n_2p<30)))*(mcal_T_2p<10)*(mcal_s2n_2p<1000)*(mcal_s2n_2p>10)*(mcal_T_ratio_2p>0.5)
        mask_2m = (~((mcal_T_2m>2)*(mcal_s2n_2m<30)))*(mcal_T_2m<10)*(mcal_s2n_2m<1000)*(mcal_s2n_2m>10)*(mcal_T_ratio_2m>0.5)

        R11i_1p_err_sum += np.sum((mcal_g_1p[:,0][mask0]-R11i_1p_sum/mask0_count)**2)
        R11i_1m_err_sum += np.sum((mcal_g_1m[:,0][mask0]-R11i_1m_sum/mask0_count)**2)
        R22i_2p_err_sum += np.sum((mcal_g_2p[:,1][mask0]-R22i_2p_sum/mask0_count)**2)
        R22i_2m_err_sum += np.sum((mcal_g_2m[:,1][mask0]-R22i_2m_sum/mask0_count)**2)

        R11s_1p_err_sum += np.sum((mcal_g_noshear[:,0][mask_1p]-R11s_1p_sum/mask_1p_count)**2)
        R11s_1m_err_sum += np.sum((mcal_g_noshear[:,0][mask_1m]-R11s_1m_sum/mask_1m_count)**2)
        R22s_2p_err_sum += np.sum((mcal_g_noshear[:,1][mask_2p]-R22s_2p_sum/mask_2p_count)**2)
        R22s_2m_err_sum += np.sum((mcal_g_noshear[:,1][mask_2m]-R22s_2m_sum/mask_2m_count)**2)

        R11_1p_err_sum += np.sum((mcal_g_1p[:,0][mask_1p]-R11_1p_sum/mask_1p_count)**2)
        R11_1m_err_sum += np.sum((mcal_g_1m[:,0][mask_1m]-R11_1m_sum/mask_1m_count)**2)
        R22_2p_err_sum += np.sum((mcal_g_2p[:,1][mask_2p]-R22_2p_sum/mask_2p_count)**2)
        R22_2m_err_sum += np.sum((mcal_g_2m[:,1][mask_2m]-R22_2m_sum/mask_2m_count)**2)

100%|███████████████████████████████████████████████████████████████████████████████| 108/108 [1:19:38<00:00, 44.25s/it]


In [17]:
R11tot_err = np.sqrt( R11i_1p_err_sum/(mask0_count**2)/dgamma**2\
                     +R11i_1m_err_sum/(mask0_count**2)/dgamma**2\
                     +R11s_1p_err_sum/(mask_1p_count**2)/dgamma**2\
                     +R11s_1m_err_sum/(mask_1m_count**2)/dgamma**2)

R22tot_err = np.sqrt( R22i_2p_err_sum/(mask0_count**2)/dgamma**2\
                     +R22i_2m_err_sum/(mask0_count**2)/dgamma**2\
                     +R22s_2p_err_sum/(mask_2p_count**2)/dgamma**2\
                     +R22s_2m_err_sum/(mask_2m_count**2)/dgamma**2)

R11_err = np.sqrt( R11_1p_err_sum/(mask_1p_count**2)/dgamma**2\
                  +R11_1m_err_sum/(mask_1m_count**2)/dgamma**2)

R22_err = np.sqrt( R22_2p_err_sum/(mask_2p_count**2)/dgamma**2\
                  +R22_2m_err_sum/(mask_2m_count**2)/dgamma**2)

In [18]:
print(R11tot_err,R22tot_err,R11_err,R22_err)

0.0026727755066305566 0.0026734576057785325 0.0018903318043964235 0.0018908369756403206
